In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

from barriers import reflect, inter2planes, linepoint, intersection, quadratic, tangenttosphere, interlinesphere, reflectmod
from barriers import randsphere, multrandsphere

pi = np.pi
sin = np.sin
cos = np.cos

In [ ]:
#The goal of this notebook is to look at the diffusion of particles in matrices of spheres.  Ideally, I would create a
#code that would generate as many spheres as I wanted spaced in a cubic fashion.  As my system relies on multiple if
#statements, I don't know how doable this is.  I am going to start with a simple code that looks at diffusion around
#two, four, and six spheres.

In [ ]:
def randoutsphere(b, s, f, p, center, radius):
    """
    Builds a single random trajectory that encounters a sphere centered at "center" and of radius "radius." This code
    is essentially the same as randsphere, but I had to modify the way the code interacted with the surface, especially
    for spheres of large radius.  I may need to go back and modify randsphere as well.

    b: base magnitude of single step
    s: variation in step size
    f: number of frames or steps to Takes
    p: particle number (should be 1 for now)
    center: point defining center of the sphere (x, y, z)
    ppoi: a point that falls on the plane of interest as a 3-element numpy array

    Output:
    0 particle number
    1 time or frames
    2 x movement
    3 angle 1 (not used)
    4 angle 2 (note used)
    5 x coordinate
    6 y coordinate
    7 z coordinate
    8 centered x coordinate
    9 centered y coordinate
    10 centered z coordinate
    11 MSD
    12 2D xy MSD
    13 2D xz MSD
    14 2D yz MSD
    15 Diffusion Coefficient (Deff)
    16 2D xy Deff
    17 2D xz Deff
    18 2D yz Deff
    19 y movement
    20 z movement
    """

    base = b
    step = s
    pi = 3.14159265359
    frames = f

    ttraject = np.zeros((frames, 22))
    ttraject[0, 0] = p

    for num in range(1, frames):

        # Create particle number
        ttraject[num, 0] = p

        # Create frame
        ttraject[num, 1] = 1 + ttraject[num-1, 1]

        # Create magnitude vectors
        ttraject[num, 2] = base*(random.random()-0.5)
        ttraject[num, 19] = base*(random.random()-0.5)
        ttraject[num, 20] = base*(random.random()-0.5)

        # Build preliminary trajectories
        ttraject[num, 5] = ttraject[num-1, 5] + ttraject[num, 2]
        ttraject[num, 6] = ttraject[num-1, 6] + ttraject[num, 19]
        ttraject[num, 7] = ttraject[num-1, 7] + ttraject[num, 20]

        # Test to see if trajectories cross boundary
        p0 = np.array([ttraject[num-1, 5], ttraject[num-1, 6], ttraject[num-1, 7]])
        p1 = np.array([ttraject[num, 5], ttraject[num, 6], ttraject[num, 7]])
        pc = center

        test1 = (p0[0] - pc[0])**2 + (p0[1] - pc[1])**2 + (p0[2] - pc[2])**2 < radius**2
        test2 = (p1[0] - pc[0])**2 + (p1[1] - pc[1])**2 + (p1[2] - pc[2])**2 < radius**2
        test3 = test2 == test1

        if test3 == True:
            # In this case, the trajectories remain unmodified
            tor = 1
        else:
            pint = interlinesphere(p0, p1, pc, radius)
            pint, v = tangenttosphere(pint, pc, radius)
            newp = reflectmod(p0, p1, v, pint)
            ttraject[num, 5] = 1.0004*(pint[0] - pc[0]) + pc[0]
            ttraject[num, 6] = 1.0004*(pint[1] - pc[1]) + pc[1]
            ttraject[num, 7] = 1.0004*(pint[2] - pc[2]) + pc[2]

    particle = ttraject[:, 0]
    time = ttraject[:, 1]
    x = ttraject[:, 5]
    y = ttraject[:, 6]
    z = ttraject[:, 7]

    ttraject[:, 8] = x - ((max(x)+min(x))/2)
    cx = ttraject[:, 8]
    ttraject[:, 9] = y - ((max(y)+min(y))/2)
    cy = ttraject[:, 9]
    ttraject[:, 10] = z - ((max(z)+min(z))/2)
    cz = ttraject[:, 10]

    # Calculate MSDs and Deffs
    for num in range(1, frames):

        ttraject[num, 11] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2 +
                             (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 12] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)
        ttraject[num, 13] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 14] = ((ttraject[num, 10]-ttraject[0, 10])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)

        ttraject[num, 15] = ttraject[num, 11]/(6*ttraject[num, 1])
        ttraject[num, 16] = ttraject[num, 12]/(4*ttraject[num, 1])
        ttraject[num, 17] = ttraject[num, 13]/(4*ttraject[num, 1])
        ttraject[num, 18] = ttraject[num, 14]/(4*ttraject[num, 1])

    MSD = ttraject[:, 11]
    MSDxy = ttraject[:, 12]
    MSDxz = ttraject[:, 13]
    MSDyz = ttraject[:, 14]

    Deff = ttraject[:, 15]
    Deffxy = ttraject[:, 16]
    Deffxz = ttraject[:, 17]
    Deffyz = ttraject[:, 18]

    return ttraject

In [ ]:
def multrandoutsphere(b, s, f, p, center, radius):
    """
    See randoutsphere
    """

    parts = p
    one = randoutsphere(b, s, f, 1, center, radius)
    counter = 1

    while counter < p + 1:
        counter = counter + 1
        one = np.append(one, randoutsphere(b, s, f, counter, center, radius), axis=0)

    return one

In [ ]:
#To experiment with the effects of one sphere, I can look at both radius of the sphere at a fixed distance as well as 
#distance from the sphere with a fixed radius
#Test distance with radius 30: 5, 10, 20, 40
#Test radius with distance 5: 5, 10, 20, 40

c1 = (-70, 0, 0)
r1 = 30
thr4 = multrandoutsphere(5, 1, 120, 1000, c1, r1)

In [ ]:
plot_3Doverlay(thr4, 0, 5, 0, 'sphere_distance4', 40, 40, 40)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr4, 0, 1, 11, 0, 0, 'MSD (um^2)', 'MSDS_sphere_distance4', 121, 801, 20, 200)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr4, 0, 1, 15, 0, 1, 'Deff (um^2/s)', 'Deff_sphere_distance4', 121, 1.6, 20, 0.5)
plt.show()

In [ ]:
def randtwospheres(b, s, f, p, c1, r1, c2, r2):
    """
    Builds a single random trajectory that encounters two spheres or radii r1 and r2 centered at c1 and c2 res-
    pectively.  When defining these spheres, make sure they don't overlap.  It could cause the code to malfunction.
    Furthermore, note that the particle starts diffusing at (0, 0, 0), and the particle cannot start inside the sphere
    or the code will malfunction.

    b: base magnitude of single step
    s: variation in step size
    f: number of frames or steps to Takes
    p: particle number (should be 1 for now)
    c1: center of the first sphere
    c2: center of the second sphere
    r1: radius of the first sphere
    r2: radius of the second sphere.

    Output:
    0 particle number
    1 time or frames
    2 x movement
    3 angle 1 (not used)
    4 angle 2 (note used)
    5 x coordinate
    6 y coordinate
    7 z coordinate
    8 centered x coordinate
    9 centered y coordinate
    10 centered z coordinate
    11 MSD
    12 2D xy MSD
    13 2D xz MSD
    14 2D yz MSD
    15 Diffusion Coefficient (Deff)
    16 2D xy Deff
    17 2D xz Deff
    18 2D yz Deff
    19 y movement
    20 z movement
    """

    base = b
    step = s
    pi = 3.14159265359
    frames = f

    ttraject = np.zeros((frames, 22))
    ttraject[0, 0] = p
    
    for num in range(1, frames):

        # Create particle number
        ttraject[num, 0] = p
        
        # Create frame
        ttraject[num, 1] = 1 + ttraject[num-1, 1]
        
        # Create magnitude vectors
        ttraject[num, 2] = base*(random.random()-0.5)
        ttraject[num, 19] = base*(random.random()-0.5)
        ttraject[num, 20] = base*(random.random()-0.5)

        # Build preliminary trajectories
        ttraject[num, 5] = ttraject[num-1, 5] + ttraject[num, 2]
        ttraject[num, 6] = ttraject[num-1, 6] + ttraject[num, 19]
        ttraject[num, 7] = ttraject[num-1, 7] + ttraject[num, 20]
        
        # Test to see if trajectories cross boundary
        p0 = np.array([ttraject[num-1, 5], ttraject[num-1, 6], ttraject[num-1, 7]])
        p1 = np.array([ttraject[num, 5], ttraject[num, 6], ttraject[num, 7]])
        
        test1 = (p0[0] - c1[0])**2 + (p0[1] - c1[1])**2 + (p0[2] - c1[2])**2 < r1**2
        test2 = (p1[0] - c1[0])**2 + (p1[1] - c1[1])**2 + (p1[2] - c1[2])**2 < r1**2
        test3 = test2 == test1
        
        test4 = (p0[0] - c2[0])**2 + (p0[1] - c2[1])**2 + (p0[2] - c2[2])**2 < r2**2
        test5 = (p1[0] - c2[0])**2 + (p1[1] - c2[1])**2 + (p1[2] - c2[2])**2 < r2**2
        test6 = test5 == test4
        
        if test3 == False:
            pint = interlinesphere(p0, p1, c1, r1)
            pint, v = tangenttosphere(pint, c1, r1)
            newp = reflectmod(p0, p1, v, pint)
            ttraject[num, 5] = 1.0004*(pint[0] - c1[0]) + c1[0]
            ttraject[num, 6] = 1.0004*(pint[1] - c1[1]) + c1[1]
            ttraject[num, 7] = 1.0004*(pint[2] - c1[2]) + c1[2]
            
        elif test6 == False:
            pint = interlinesphere(p0, p1, c2, r2)
            pint, v = tangenttosphere(pint, c2, r2)
            newp = reflectmod(p0, p1, v, pint)
            ttraject[num, 5] = 1.0004*(pint[0] - c2[0]) + c2[0]
            ttraject[num, 6] = 1.0004*(pint[1] - c2[1]) + c2[1]
            ttraject[num, 7] = 1.0004*(pint[2] - c2[2]) + c2[2]

        else:
            #In this case, the trajectories remain unmodified
            tor = 1

    particle = ttraject[:, 0]
    time = ttraject[:, 1]
    x = ttraject[:, 5]
    y = ttraject[:, 6]
    z = ttraject[:, 7]

    ttraject[:, 8] = x - ((max(x)+min(x))/2)
    cx = ttraject[:, 8]
    ttraject[:, 9] = y - ((max(y)+min(y))/2)
    cy = ttraject[:, 9]
    ttraject[:, 10] = z - ((max(z)+min(z))/2)
    cz = ttraject[:, 10]
    
    # Calculate MSDs and Deffs
    for num in range(1, frames):

        ttraject[num, 11] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2 +
                             (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 12] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)
        ttraject[num, 13] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 14] = ((ttraject[num, 10]-ttraject[0, 10])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)

        ttraject[num, 15] = ttraject[num, 11]/(6*ttraject[num, 1])
        ttraject[num, 16] = ttraject[num, 12]/(4*ttraject[num, 1])
        ttraject[num, 17] = ttraject[num, 13]/(4*ttraject[num, 1])
        ttraject[num, 18] = ttraject[num, 14]/(4*ttraject[num, 1])

    MSD = ttraject[:, 11]
    MSDxy = ttraject[:, 12]
    MSDxz = ttraject[:, 13]
    MSDyz = ttraject[:, 14]

    Deff = ttraject[:, 15]
    Deffxy = ttraject[:, 16]
    Deffxz = ttraject[:, 17]
    Deffyz = ttraject[:, 18]

    return ttraject

In [ ]:
def multrandtwospheres(b, s, f, p, c1, r1, c2, r2):
    """
    See randsphere
    """

    parts = p
    one = randtwospheres(b, s, f, 1, c1, r1, c2, r2)
    counter = 1

    while counter < p + 1:
        counter = counter + 1
        one = np.append(one, randtwospheres(b, s, f, counter, c1, r1, c2, r2), axis=0)

    return one

In [ ]:
#Test distance with radius 30: 5, 10, 20, 40
#Test radius with distance 5: 5, 10, 20, 40

c1 = (-45, 0, 0)
c2 = (-45, 0, 0)
r1 = 40
r2 = 40


thr5 = multrandtwospheres(5, 1, 120, 1000, c1, r1, c2, r2)
plot_3Doverlay(thr5, 0, 5, 0, 'twosphere_radius4', 50, 50, 50)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 11, 0, 0, 'MSD (um^2)', 'MSDS_twosphere_radius4', 121, 601, 20, 200)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 15, 0, 1, 'Deff (um^2/s)', 'Deff_twosphere_radius4', 121, 1.2, 20, 0.5)
plt.show()

In [ ]:
#I need to write a code to generate a dictionary of centerpoints of spheres.  You will be able to put in number
#along one axis e.g. inputting 3 will yield 27 spheres.  However, since particles start diffusing at 0, odd numbers are
#out.  Only even numbers.  You should also be able to determine the spacing.

def centers(rows, spaces):
    

In [ ]:
nx, ny, nz = (4, 4, 4)
x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
z = np.linspace(-1, 1, nz)

x

In [ ]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
z = np.sin(xx**2 + yy**2)/(xx**2 + yy**2)
h = plt.contourf(x, y, z)
plt.show()

In [ ]:
X, Y = np.mgrid[-5:5.1:0.5, -5:5.1:0.5]
X

In [ ]:
xyz = np.mgrid[-10:10.1:20, -10:10.1:20, -10:10.1:20].reshape(3, -1).T

In [4]:
def grid(spaces, number):
    x = spaces
    y = x + 0.1
    xn = number
    xn2 = 2 * x / (xn-1)
    
    xyz = np.mgrid[-x:y:xn2, -x:y:xn2, -x:y:xn2].reshape(3, -1).T
    return xyz

In [16]:
def randmultspheres(b, s, f, p, c, r):
    """
    Builds a single random trajectory that encounters two spheres or radii r1 and r2 centered at c1 and c2 res-
    pectively.  When defining these spheres, make sure they don't overlap.  It could cause the code to malfunction.
    Furthermore, note that the particle starts diffusing at (0, 0, 0), and the particle cannot start inside the sphere
    or the code will malfunction.

    b: base magnitude of single step
    s: variation in step size
    f: number of frames or steps to Takes
    p: particle number (should be 1 for now)
    c: array of xyz coordinates of spheres to be included
    r: radius of spheres

    Output:
    0 particle number
    1 time or frames
    2 x movement
    3 angle 1 (not used)
    4 angle 2 (note used)
    5 x coordinate
    6 y coordinate
    7 z coordinate
    8 centered x coordinate
    9 centered y coordinate
    10 centered z coordinate
    11 MSD
    12 2D xy MSD
    13 2D xz MSD
    14 2D yz MSD
    15 Diffusion Coefficient (Deff)
    16 2D xy Deff
    17 2D xz Deff
    18 2D yz Deff
    19 y movement
    20 z movement
    """

    base = b
    step = s
    pi = 3.14159265359
    frames = f

    ttraject = np.zeros((frames, 22))
    ttraject[0, 0] = p
    
    for num in range(1, frames):

        # Create particle number
        ttraject[num, 0] = p
        
        # Create frame
        ttraject[num, 1] = 1 + ttraject[num-1, 1]
        
        # Create magnitude vectors
        ttraject[num, 2] = base*(random.random()-0.5)
        ttraject[num, 19] = base*(random.random()-0.5)
        ttraject[num, 20] = base*(random.random()-0.5)

        # Build preliminary trajectories
        ttraject[num, 5] = ttraject[num-1, 5] + ttraject[num, 2]
        ttraject[num, 6] = ttraject[num-1, 6] + ttraject[num, 19]
        ttraject[num, 7] = ttraject[num-1, 7] + ttraject[num, 20]
        
        # Test to see if trajectories cross boundary
        p0 = np.array([ttraject[num-1, 5], ttraject[num-1, 6], ttraject[num-1, 7]])
        p1 = np.array([ttraject[num, 5], ttraject[num, 6], ttraject[num, 7]])
        
        co = 0
        centers = np.shape(c)[0] - 1
        test1 = dict()
        test2 = dict()
        test3 = dict()
        
        while co <= centers:
            test1[co] = (p0[0] - c[co][0])**2 + (p0[1] - c[co][1])**2 + (p0[2] - c[co][2])**2 < r**2
            test2[co] = (p1[0] - c[co][0])**2 + (p1[1] - c[co][1])**2 + (p1[2] - c[co][2])**2 < r**2
            test3[co] = test2[co] == test1[co]
            co = co + 1
        
        co = 0
        
        while co <= centers:
        
            if test3[co] == False:
                pint = interlinesphere(p0, p1, c[co], r)
                pint, v = tangenttosphere(pint, c[co], r)
                ttraject[num, 5] = 1.00004*(pint[0] - c[co][0]) + c[co][0]
                ttraject[num, 6] = 1.00004*(pint[1] - c[co][1]) + c[co][1]
                ttraject[num, 7] = 1.00004*(pint[2] - c[co][2]) + c[co][2]

            co = co + 1

    particle = ttraject[:, 0]
    time = ttraject[:, 1]
    x = ttraject[:, 5]
    y = ttraject[:, 6]
    z = ttraject[:, 7]

    ttraject[:, 8] = x - ((max(x)+min(x))/2)
    cx = ttraject[:, 8]
    ttraject[:, 9] = y - ((max(y)+min(y))/2)
    cy = ttraject[:, 9]
    ttraject[:, 10] = z - ((max(z)+min(z))/2)
    cz = ttraject[:, 10]
    
    # Calculate MSDs and Deffs
    for num in range(1, frames):

        ttraject[num, 11] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2 +
                             (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 12] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)
        ttraject[num, 13] = ((ttraject[num, 8]-ttraject[0, 8])**2 + (ttraject[num, 10]-ttraject[0, 10])**2)
        ttraject[num, 14] = ((ttraject[num, 10]-ttraject[0, 10])**2 + (ttraject[num, 9]-ttraject[0, 9])**2)

        ttraject[num, 15] = ttraject[num, 11]/(6*ttraject[num, 1])
        ttraject[num, 16] = ttraject[num, 12]/(4*ttraject[num, 1])
        ttraject[num, 17] = ttraject[num, 13]/(4*ttraject[num, 1])
        ttraject[num, 18] = ttraject[num, 14]/(4*ttraject[num, 1])

    MSD = ttraject[:, 11]
    MSDxy = ttraject[:, 12]
    MSDxz = ttraject[:, 13]
    MSDyz = ttraject[:, 14]

    Deff = ttraject[:, 15]
    Deffxy = ttraject[:, 16]
    Deffxz = ttraject[:, 17]
    Deffyz = ttraject[:, 18]

    return ttraject

In [17]:
def multrandmultspheres(b, s, f, p, c, r):
    """
    See randmultspheres
    """

    parts = p
    one = randmultspheres(b, s, f, 1, c, r)
    counter = 1

    while counter < p + 1:
        counter = counter + 1
        one = np.append(one, randmultspheres(b, s, f, counter, c, r), axis=0)

    return one

In [71]:
xyz = grid(300, 76)
xyz[0:100, :]

array([[-300., -300., -300.],
       [-300., -300., -292.],
       [-300., -300., -284.],
       [-300., -300., -276.],
       [-300., -300., -268.],
       [-300., -300., -260.],
       [-300., -300., -252.],
       [-300., -300., -244.],
       [-300., -300., -236.],
       [-300., -300., -228.],
       [-300., -300., -220.],
       [-300., -300., -212.],
       [-300., -300., -204.],
       [-300., -300., -196.],
       [-300., -300., -188.],
       [-300., -300., -180.],
       [-300., -300., -172.],
       [-300., -300., -164.],
       [-300., -300., -156.],
       [-300., -300., -148.],
       [-300., -300., -140.],
       [-300., -300., -132.],
       [-300., -300., -124.],
       [-300., -300., -116.],
       [-300., -300., -108.],
       [-300., -300., -100.],
       [-300., -300.,  -92.],
       [-300., -300.,  -84.],
       [-300., -300.,  -76.],
       [-300., -300.,  -68.],
       [-300., -300.,  -60.],
       [-300., -300.,  -52.],
       [-300., -300.,  -44.],
       [-3

In [39]:
thr5 = multrandmultspheres(40, 1, 100, 1000, xyz[0:108, :], 43)
plot_3Doverlay(thr5, 0, 5, 0, 'multspheres3_3', 400, 400, 400)
plt.show()

In [ ]:
plot_3Doverlay(thr5, 0, 5, 0, 'multspheres1_5', 400, 400, 400)
plt.show()

In [40]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 11, 0, 0, 'MSD (um^2)', 'MSDS_multspheres3_3', 101, 40001, 20, 8000)
plt.show()

In [41]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 15, 0, 1, 'Deff (um^2/s)', 'Deff_multspheres3_3', 101, 100, 20, 20)
plt.show()

In [ ]:
#Compare with isotropic behavior:
thr = multrandanisorot(40, 1, 100, 1000, 1, 1, 1, 0, 0, 0)
plot_3Doverlay(thr, 0, 5, 0, 'iso_3Dplot', 400, 400, 400)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 11, 0, 0, 'MSD (um^2)', 'MSDS_iso1', 101, 40001, 20, 8000)
plt.show()

In [ ]:
one = plot_Mean2DMSDsorDeff(thr5, 0, 1, 15, 0, 1, 'Deff (um^2/s)', 'Deff_iso1', 101, 100, 20, 20)
plt.show()